In [121]:
#Code to cluster named-entities into clusters that contain variations of the name for same entity.
#For example, Donald Trump, Donal J Trump, D Trump etc.
#Resulting NE clusters are store as a pickle (Json format)

import pandas as pd
import pickle
import sys
import numpy as np
import os
import re
from os import listdir
from os.path import isfile, join
import timeit
import spacy
from nltk import word_tokenize
from nltk.corpus import stopwords
from itertools import combinations

from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
import sklearn

nlp = spacy.load('en_core_web_lg')
stop_words = set(stopwords.words('english')) 


In [375]:
path = "Nammed_entities/"
files = [f for f in listdir(path) if isfile(join(path, f))]
#print(len(files))

In [376]:
def add_to_dict(final_dict, new_element):
    final_keys = final_dict.keys()
    new_keys = new_element.keys()
    #print(final_keys, new_keys)
    for key in new_keys:
        if(key not in final_keys):
            final_dict[key] = new_element[key]
        else:
            #print("here")
            for element in new_element[key]:
                if(isinstance(element, str)):
                    #print(element, type(element))
                    final_dict[key].add(element)
                
            #final_keys[key].append(element)
    #for index, item in enumerate(new_element):
    
    return final_dict

#print(len(final_dict['ORG']), len(vector_dict['ORG']), vector_dict['ORG'][2].shape)
def remove_stopwords(tokens):
    
    token = word_tokenize(tokens)
    l = [tok for tok in token if tok not in stop_words]
    #print(l)    
    return " ".join(l)


def get_final_dict(files):    
    
    final_dict = dict()
    for index, fl in enumerate(files):
        if(index%1000 == 0):
            start = timeit.default_timer()

        with open(path+fl, 'rb') as f:
            d = pickle.load(f)
            final_dict = add_to_dict(final_dict, d)

        if(index%1000 == 0):
            end = timeit.default_timer()
            est = (1-(index/len(files))) * (end-start)
            #print("Estimated time left {:2.1%}".format(est), end="\r")
            #print("Estimated time left : ", index, "\t", est)

            #sys.exit()
            
    return final_dict


def mean_vectorizor(model, tokenized_sent, dim):
    
    #for key in final_dict:
    c = len(tokenized_sent)
    print("c = ", c, "\t", tokenized_sent)
    v = np.zeros(300)
    for tok in tokenized_sent:
        if(tok in model.wv.vocab):
            v += model[tok]
            print(model[tok])
    
    return v/c
            
def get_fasttext_vectors(model, final_dict):
    
    vector_dict = dict()
    for key in final_dict:
        #print("Key : ", key)
        #print(len(final_dict[key]))
        #print(len(model.wv.vocab))
        vec = []
        #if False not in [s.isalpha() for s in st.split()]:
        #l = [model[w.lower()] for w in final_dict[key] if all((ord(char) >= 97 and ord(char) <= 122) or (ord(char) >= 65 and ord(char) <= 122) for char in w)]
        l = []
        for entity in final_dict[key]:
            if(all((ord(char) >= 97 and ord(char) <= 122) or (ord(char) >= 65 and ord(char) <= 90) or ord(char) == 32 for char in entity)):
                if(entity.lower() in model.wv.vocab):
                    #print("entity ", entity)
                    l.append(model[entity.lower()])
                else:
                    l.append(np.zeros(300))
            else:
                l.append(np.zeros(300))
        
        vector_dict[key] = l
        
    return vector_dict

def normalize_text(s):
    """
    Given a text, cleans and normalizes it. Feel free to add your own stuff.
    """
    s = s.lower()
    # Replace ips
    s = re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', ' _ip_ ', s)
    # Isolate punctuation
    s = re.sub(r'([\'\"\.\(\)\!\?\-\\\/\,])', r' \1 ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Replace numbers and symbols with language
    s = s.replace('&', ' and ')
    s = s.replace('@', ' at ')
    s = s.replace('0', ' zero ')
    s = s.replace('1', ' one ')
    s = s.replace('2', ' two ')
    s = s.replace('3', ' three ')
    s = s.replace('4', ' four ')
    s = s.replace('5', ' five ')
    s = s.replace('6', ' six ')
    s = s.replace('7', ' seven ')
    s = s.replace('8', ' eight ')
    s = s.replace('9', ' nine ')
    
    return s
        
def get_fasttext_vecs(model, text):
    
    text_tokens = word_tokenize(text)
    vecs = []
    for token in text_tokens:
        
        token = normalize_text(token)
        if(token in model):
            vecs.append(model[token])
    
    vecs = np.array(vecs)
    return np.mean(vecs, axis=0)
    
def get_word_vectors(final_dict):

    vector_dict = dict()
    for key in final_dict:
        print("Key : ", key)
        print(len(final_dict[key]))
        l = [nlp(x).vector for x in final_dict[key]]
        print(len(l))
        vector_dict[key] = l
        
    return vector_dict

def update_centroids(cls_vectors):
    
    centroids = dict()
    for key in cls_vectors:
        #print(len(cls_vectors[key]))
        #sys.exit()
        center = np.mean(np.array(cls_vectors[key]), axis=0)
        centroids[key] = center
    
    return centroids

def cluster_entity(entities, entities_list, centroids, clusters, cls_vectors):
    
    eps = 1.50
    lr = 0.10
       
    keys = centroids.keys()
    center_values = centroids.values()
    counti = 0
    countf = 2
    iter = 0
    while((countf - counti >= 2 and len(entities_list) > 0) or iter > 10):
        counti = countf
        iter += 1
        for index, ent in enumerate(entities):
            #if(index == 28):
            #print(entities)
            #print()
            #print(center_values)
            #print(type(ent))

            if(not np.all(np.isnan(ent))):
                dist = np.array([np.linalg.norm(ent-center) for center in center_values])
                #print(index, "\t", dist[0:10])
                imax = np.nanargmin(dist)
                if(dist[imax] < 1.50):
                    clusters[imax].append(entities_list[index])
                    cls_vectors[imax].append(ent)
                    cen = np.mean(np.array(cls_vectors[imax]), axis=0)
                    centroids[imax] = cen
                    del entities[index]
                    del entities_list[index]
                    countf+=1

        print("New elements added to cluster : ", countf, "\tNumber of clusters ", len(clusters))

    return clusters, cls_vectors


def cluster_entities(vector_dict, final_dict):

    cluster = DBSCAN(eps=2.8, min_samples=2)
    #cluster = MeanShift(bandwidth=1)
    for key in vector_dict:
        key = 'PERSON'
        vect = np.array(vector_dict[key])
        print("key ", key)
        print(len(vect), vect.shape)
        entity = list(final_dict[key])
        print(" -------------------- ")
        print(len(entity))
        entity = np.array(entity)
        
        pred = cluster.fit_predict(vect)
        labels = cluster.labels_
        print("Number of entities : ", vect.shape[0], " Number fo clusters ", len(set(labels)))
        print(labels)
        print(cluster)
        print("Pred ", pred)


        lab = set(pred)
        for i in lab:
            print("cluster ", i)
            index = np.where(pred == i)
            print(index)
            for j in index:
                print("j ", j)
                print(entity[j])
                print("\n")
                #for h in [(head, ind) for head, ind in zip(headline[j], index[0])]:
                #    print(h)


def get_bigram_entities(entities):
    
    bigram = []
    for ent in entities:
        sname = ent.split()
        if(len(sname) == 2):
            bigram.append(ent)
    
    return bigram


def get_name_variations(name):
    
    sname = name.split()
    abv_name = [s[0] for s in sname]
    #print(abv_name)
    l = len(sname)
    rng = list(range(l))
    #print(rng)
    new_names = []
    for i, s in enumerate(sname):
        comb = list(combinations(rng, i))
        for c in comb:
            new_name = dict()
            #print("c is ", c)
            non_abv_index = set(rng) - set(c)
            for abv_index in c:
                #print("----------------\nhere\n", abv_index, non_abv_index)
                new_name[abv_index] = abv_name[abv_index]
            for index in non_abv_index:
                new_name[index] = sname[index]
            
            f = [new_name[key] for key in new_name]
            x = " ".join(f)
            new_names.append(x)
            #print(x)
            
    return new_names
        
def get_clustered_names(final_dict):
    
    clusters = []
    entities = final_dict['PERSON']
    bigrams = get_bigram_entities(entities)
    for entity in bigrams:
        names = get_name_variations(entity)
        clusters.append(names)
        
    return clusters
        

In [ ]:
from gensim.models.wrappers import FastText
from gensim.models import KeyedVectors

model = FastText.load_fasttext_format('../word_embedings/wiki.en.bin')
#model = KeyedVectors.load_word2vec_format('../word_embedings/wiki.en.vec')

In [377]:
final_dict = get_final_dict(files)
processed_dict = dict()
for key in final_dict.keys():
    processed_dict[key] = [remove_stopwords(tok) for tok in final_dict[key]]
 
print(processed_dict.keys())
vector_dict = get_fasttext_vectors(model, final_dict)

cls_vectors = dict() 
for index, cls in enumerate(clusters):
    vec = [get_fasttext_vecs(model, cl) for cl in cls]
    cls_vectors[index] = vec
 
entities = final_dict['PERSON']
non_bigrams_ent = [ent for ent in entities if len(ent.split()) != 2]
print(len(entities), len(non_bigrams_ent))
#sys.exit()

nobigrams_entity_vectors = [get_fasttext_vecs(model, ent) for ent in non_bigrams_ent]

print(len(cls_vectors), len(cls_vectors[0]))
print(len(clusters), type(clusters))
print(len(nobigrams_entity_vectors), len(nobigrams_entity_vectors[0]))

#vector_dict = get_word_vectors(processed_dict)
#entity = np.array([remove_stopwords(final_dict[key]) for key in final_dict])

entities = final_dict['PERSON']
centroids = update_centroids(cls_vectors)
clss, cls_vec = cluster_entity(nobigrams_entity_vectors, non_bigrams_ent, centroids, clusters, cls_vectors)   

final_entity_clusters = dict()
print(type(clss), len(clss))
for index, cl in enumerate(clss):
    #print(index)
    final_entity_clusters[index] = set(cl)

filepath = "Nammed_entities/clusters_ne_.pickle"
with open(filepath, 'wb') as f:
    vd = pickle.dump(final_entity_clusters, f)


dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

/home/ahaque2/project/virtual_environment_1/lib/python3.5/site-packages/ipykernel_launcher.py:77: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


133788 52399
3153 4
3153 <class 'list'>
52399 300
New elements added to cluster :  1467 	Number of clusters  3153
New elements added to cluster :  1660 	Number of clusters  3153
New elements added to cluster :  1721 	Number of clusters  3153
New elements added to cluster :  1730 	Number of clusters  3153
New elements added to cluster :  1732 	Number of clusters  3153
New elements added to cluster :  1732 	Number of clusters  3153
<class 'list'> 3153


In [378]:
filepath = "Nammed_entities/clusters_ne_.pickle"
vd = None
with open(filepath, 'rb') as f:
    vd = pickle.load(f)

In [379]:
print(len(vd))
#print(vd.keys())
for key in vd:
    if(len(vd[key]) > 4):
        print(vd[key])

3153
{'Jonathan G', 'Jonathan A. Greenblatt', 'J Greenblatt', 'Jonathan A. Greenblat', 'Jonathan Greenblatt'}
{'Don Lemon', 'D Lemon', 'This Don Lemon', 'Don L', 'Don Lemon at night'}
{'Boy S', 'the Girl Scouts or Boy Scouts', 'the Boy Scouts of America', 'the Girl Scouts USA', 'the Boy Scouts', 'the Girl Scouts', 'The Boy Scouts', 'the Boy Scout   codes', 'Boy Scouts', 'B Scouts', 'a Boy Scout', 'The Girl Scouts', 'Girl Scouts meet'}
{'Oprah Winfrey State', 'Oprah W', 'Oprah Winfrey Network', 'O Winfrey', 'Oprah Gail Winfrey', 'Oprah Winfrey Show', 'Oprah Winfrey'}
{'Michael Cohen', 'M Cohen', 'Michael D. Cohen', 'Michael C', 'Michael L. Cohen'}
{'R Kelly', 'Robert Kelly Otiendo', 'Robert Kelly', 'Robert Michael Kelly', 'Robert K', 'Robert M. Kelly'}
{'Jay Z. It', 'Jay Z. This', 'J Z', 'Jay Z. Hey', 'Jay Z. Lines', 'Jay Z', 'Jay Z.  Jay Z'}
{'Nicholas M. O’Donnell', 'James O’Toole Jr.', 'A O’Neill', 'Ann O’Neill', 'Thomas P. O’Neill Jr.', 'John P. O’Donnell', 'Eileen O’Farrell Smith',

In [37]:
def disambiguate_names(final_dict):
    
    
    import disamby
    import disamby.preprocessors as pre
    from disamby import Disamby
    
    entities = list(final_dict['PERSON'])
    
    print(len(entities))
    
    df = pd.DataFrame({
    'name': entities})

    
    pipeline = [
    pre.normalize_whitespace,
    pre.remove_punctuation,
    lambda x: pre.trigram(x) + pre.split_words(x)  # any python function is allowed
    ]

    # instantiate the disamby object, it applies the given pre-processing pipeline and
    # computes their frequency.
    dis = Disamby(df, pipeline)

    # let disamby compute disambiguated sets. Node that a threshold must be given or it
    # defaults to 0.
    clusters = dis.disambiguated_sets(threshold=0.2)
    return clusters, df
    
#clusters, df = disambiguate_names(final_dict)

5618


100%|██████████| 5618/5618 [00:22<00:00, 255.03it/s]
